# Kaggle Competition: ASHRAE - Great Energy Predictor III

São feitos investimentos significativos para melhorar a eficiência de construções e reduzir custos e emissões. A questão é: as melhorias estão funcionando? Com o pagamento por desempenho, o proprietário do edifício paga com base na diferença entre o consumo real de energia e o que eles teriam usado sem nenhuma reforma. Os últimos valores têm que vir de um modelo. Os métodos atuais de estimativa são fragmentados e não escalam bem. Alguns assumem um tipo específico de medidor ou não trabalham com diferentes tipos de construção.

Nesta competição, terão de ser desenvolvidos modelos precisos do uso de energia de edifícios medidos nas seguintes áreas: medidores de água resfriada, elétrica, água quente e vapor. Os dados são provenientes de mais de 1.000 edifícios em um período de três anos (62 milhões de linhas). Com melhores estimativas desses investimentos em economia de energia, os investidores em larga escala e as instituições financeiras estarão mais inclinadas a investir nessa área para permitir o progresso de edificações eficientes. 

In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_log_error
from math import sqrt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

import random, datetime, math, gc
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

SEED = 99
random.seed(SEED)
np.random.seed(SEED)

pd.set_option('display.max_columns', None)

###### A parte de análise exploratória dos dados, e de minificação e tratamento dos dados foram realizados em outros notebooks que serão postados posteriormente durante o desenvolvimento deste projeto.

# Load data

Foram fornecidos 5 arquivos pela competição, os arquivos de treino e teste com Id’s das construções, quando as medições foram feitas e informações sobre o tipo de energia que é consumida. Os arquivos de dados climáticos para os períodos das medições de treino e teste e as informações sobre as construções. Fiz o tratamento dos dados em um notebook anterior a este, onde realizei a minificação dos dados e a junção de todos os dados em dados de treino (aproximadamente 20 milhões de linhas) e teste (aproximadamente 42 milhões de linhas). Esse notebook forneceu como saída os dois arquivos .pkl que usarei neste notebook.

In [2]:
dftrain = pd.read_pickle('train.pkl')
dftest = pd.read_pickle('test.pkl')
print ('Train: %s , Test %s' % (dftrain.shape, dftest.shape) )

Train: (20216100, 16) , Test (41697600, 16)


In [3]:
dftrain.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01 00:00:00,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,1,0,2016-01-01 00:00:00,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
2,2,0,2016-01-01 00:00:00,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
3,3,0,2016-01-01 00:00:00,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
4,4,0,2016-01-01 00:00:00,0.0,0,Education,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0


# Preprocessing

Realizo a junção dos dados de treino e teste para ser realizada a feature engineering em todos os dados juntos. Apesar deste método não ser viável em produção em muitos dos casos, no âmbito da competição é eficiente.

In [4]:
dffe = pd.concat([dftrain,dftest], axis =0, ignore_index=True, sort=False)
print('Shape :',dffe.shape)

Shape : (61913700, 17)


Preencho os valores nulos com 0 e -999

In [5]:
dffe['floor_count'] = dffe['floor_count'].fillna(0).astype(np.int8)
dffe['year_built'] = dffe['year_built'].fillna(-999).astype(np.int16)

# Feature engineering

Nesta fase desenvolvi features referentes ao tempo para contribuir com a detecção de sazonalidade pelo modelo. Também foi adicionado um dataset intrínseco do Pandas referente a feriados americanos que podem contribuir com o consumo de energia das edificações.

In [6]:
dffe['timestamp'] = pd.to_datetime(dffe['timestamp'])

In [7]:
dates_range = pd.date_range(start='2015-12-31', end='2019-01-01')
us_holidays = calendar().holidays(start=dates_range.min(), end=dates_range.max())

In [8]:
dffe['DT_MO'] = dffe['timestamp'].dt.month.astype(np.int8)
dffe['DT_WY'] = dffe['timestamp'].dt.weekofyear.astype(np.int8)
dffe['DT_DY'] = dffe['timestamp'].dt.dayofyear.astype(np.int16)
dffe['is_holiday'] = (dffe['timestamp'].dt.date.astype('datetime64').isin(us_holidays)).astype(np.int8)
dffe['DT_H'] = dffe['timestamp'].dt.hour.astype(np.int8)
dffe['DT_day_week'] = dffe['timestamp'].dt.dayofweek.astype(np.int8)
dffe['DT_day_month'] = dffe['timestamp'].dt.day.astype(np.int8)
dffe['DT_week_month'] = dffe['timestamp'].dt.day/7
dffe['DT_week_month'] = dffe['DT_week_month'].apply(lambda x: math.ceil(x)).astype(np.int8)

Fiz o logaritmo da variável target, para ter uma distribuição mais próxima da normal.

In [9]:
dffe['meter_reading'] = np.log1p(dffe['meter_reading'])

Deletei as colunas que não serão utilizadas pelo modelo.

In [10]:
dffe.drop(['timestamp','row_id'],axis=1,inplace=True)

#### Encoding

Transformei as colunas que são categóricas no tipo “category” para o Lgbm lidar melhor com estas colunas.

In [11]:
category = ['DT_MO','DT_WY','DT_DY','DT_H','DT_day_week','DT_day_month','DT_week_month','meter','primary_use','site_id','building_id']
for col in category :
    dffe[col] = dffe[col].astype('category')

# Validation

A validação foi feita replicando da melhor maneira possível a situação encontrada na competição. Na competição há 1 ano e meio de dados de treino e 1 ano e meio de dados de teste. Então eu utilizei uma técnica de holdout que funciona melhor para time series, dividi os dados de treino que estão organizados pelo tempo na metade, e usei a primeira parte para treino e a segunda para validação. As previsões foram realizadas nestes dados de treino através do metodo de validação cruzada KFold e os resultados comparados com os dados de validação do holdout. Neste setup são feitos os testes de feature engineering adicionando uma feature por vez e observando se a métrica de avaliação do modelo RMSLE melhora.

In [12]:
df_val_train = dffe[:int(dftrain.shape[0]*0.5)]
df_val_test  = dffe[int(dftrain.shape[0]*0.5):dftrain.shape[0]]
print('Shape treino: {:} | Shape teste: {:}'.format(df_val_train.shape, df_val_test.shape))
X_val_train = df_val_train.drop(['meter_reading'], axis = 1)
y_val_train  = df_val_train.meter_reading

Shape treino: (10108050, 23) | Shape teste: (10108050, 23)


In [13]:
lgb_params = {
                    'objective':'regression',
                    'metric':'rmse',
                    'n_jobs':-1,
                    'learning_rate':0.25,
                    'num_leaves': 40,
                    'max_depth':-1,
                    'subsample':0.85,
                    'n_estimators':100,
                    'seed': SEED,
                    'early_stopping_rounds':100, 
                } 

In [14]:
K = 2
folds = KFold(K, shuffle=True, random_state = SEED)
finalpred_validation = np.zeros(df_val_train.shape[0])
for fold , (train_index,test_index) in enumerate(folds.split(X_val_train, y_val_train)):
    print('Fold:',fold+1)
          
    X_traincv, X_testcv = X_val_train.iloc[train_index], X_val_train.iloc[test_index]
    y_traincv, y_testcv = y_val_train.iloc[train_index], y_val_train.iloc[test_index]
    
    train_data = lgb.Dataset(X_traincv, y_traincv)
    val_data   = lgb.Dataset(X_testcv, y_testcv)
    
    %time LGBM = lgb.train(lgb_params, train_data, valid_sets=[train_data,val_data], verbose_eval=25)
    %time finalpred_validation += np.expm1(LGBM.predict(df_val_test.drop(['meter_reading'], axis = 1)))
    print()
    
finalpred_validation /= K

Fold: 1
Training until validation scores don't improve for 100 rounds.
[25]	training's rmse: 0.879328	valid_1's rmse: 0.882023
[50]	training's rmse: 0.783415	valid_1's rmse: 0.787898
[75]	training's rmse: 0.750123	valid_1's rmse: 0.756113
[100]	training's rmse: 0.727066	valid_1's rmse: 0.734433
Did not meet early stopping. Best iteration is:
[100]	training's rmse: 0.727066	valid_1's rmse: 0.734433
Wall time: 53.6 s
Wall time: 36.1 s

Fold: 2
Training until validation scores don't improve for 100 rounds.
[25]	training's rmse: 0.879265	valid_1's rmse: 0.880188
[50]	training's rmse: 0.783023	valid_1's rmse: 0.785983
[75]	training's rmse: 0.745779	valid_1's rmse: 0.750223
[100]	training's rmse: 0.721741	valid_1's rmse: 0.727372
Did not meet early stopping. Best iteration is:
[100]	training's rmse: 0.721741	valid_1's rmse: 0.727372
Wall time: 50.4 s
Wall time: 39.6 s



In [15]:
finalpred_validation = np.where((finalpred_validation < 0), 0, finalpred_validation)
print("Root mean squared logarithm error: %.2f"
      % np.sqrt(mean_squared_log_error(df_val_test.meter_reading, finalpred_validation)))

Root mean squared logarithm error: 3.03


In [16]:
del df_val_train, df_val_test, X_val_train, y_val_train
gc.collect()

94

# Modeling

A modelagem foi realizada em um setup de validação cruzada KFold com 5 folds. O modelo faz a validação com a fold que ficou de fora em cada iteração, usando esta mesma fold para parar o treino em caso de não haver mais melhora da métrica de avaliação. Inicialmente separo de volta os dados em treino e teste a partir do dataframe que foi realizado a feature engineering. Depois divido nas features preditoras(X_train) e o target(Y_train) e faço as previsões.

#### Split

In [12]:
dftrain = dffe[:dftrain.shape[0]]
dftest  = dffe[dftrain.shape[0]:].drop(['meter_reading'], axis=1)
print('Shape treino: {:} | Shape teste: {:}'.format(dftrain.shape, dftest.shape))

Shape treino: (20216100, 23) | Shape teste: (41697600, 22)


In [13]:
X_train = dftrain.drop(['meter_reading'], axis = 1)
y_train  = dftrain.meter_reading

In [14]:
del dffe
gc.collect()

185

###### Model

In [15]:
lgb_params = {
                    'objective':'regression',
                    'metric':'rmse',
                    'n_jobs':-1,
                    'learning_rate':0.05,
                    'num_leaves': 40,
                    'max_depth':-1,
                    'subsample':0.85,
                    'n_estimators':3000,
                    'seed': SEED,
                    'early_stopping_rounds':100,
                } 

In [16]:
K = 5
folds = KFold(K, shuffle=True, random_state = SEED)
finalpred = np.zeros(dftest.shape[0])
for fold , (train_index,test_index) in enumerate(folds.split(X_train, y_train)):
    print('Fold:',fold+1)
          
    X_traincv, X_testcv = X_train.iloc[train_index], X_train.iloc[test_index]
    y_traincv, y_testcv = y_train.iloc[train_index], y_train.iloc[test_index]
    
    train_data = lgb.Dataset(X_traincv, y_traincv)
    val_data   = lgb.Dataset(X_testcv, y_testcv)
    
    %time LGBM = lgb.train(lgb_params, train_data, valid_sets= [train_data,val_data], verbose_eval=250)
    %time finalpred += np.expm1(LGBM.predict(dftest))
    print()
    
finalpred /= K

Fold: 1
Training until validation scores don't improve for 100 rounds.
[250]	training's rmse: 0.851901	valid_1's rmse: 0.852512
[500]	training's rmse: 0.776566	valid_1's rmse: 0.778414
[750]	training's rmse: 0.736921	valid_1's rmse: 0.739843
[1000]	training's rmse: 0.704201	valid_1's rmse: 0.708086
[1250]	training's rmse: 0.68217	valid_1's rmse: 0.686925
[1500]	training's rmse: 0.663228	valid_1's rmse: 0.668874
[1750]	training's rmse: 0.647885	valid_1's rmse: 0.654386
[2000]	training's rmse: 0.636522	valid_1's rmse: 0.64377
[2250]	training's rmse: 0.62622	valid_1's rmse: 0.634212
[2500]	training's rmse: 0.617329	valid_1's rmse: 0.626081
[2750]	training's rmse: 0.608467	valid_1's rmse: 0.618096
[3000]	training's rmse: 0.601309	valid_1's rmse: 0.611685
Did not meet early stopping. Best iteration is:
[3000]	training's rmse: 0.601309	valid_1's rmse: 0.611685
Wall time: 38min 21s
Wall time: 1h 3min 27s

Fold: 2
Training until validation scores don't improve for 100 rounds.
[250]	training's 

As previsões são a média do exponencial (para transformar de volta na escala original) da previsão de cada fold.

# Submission

Leio o arquivo de submissão da competição com os id's das linhas, e adiciono as minhas previsões. Salvo o arquivo em formato comprimido para economizar espaço no disco e para o upload no site do Kaggle ser mais rápido.

In [17]:
submission = pd.read_csv('sample_submission.csv')
submission['meter_reading'] = finalpred
submission['meter_reading'] = np.where((submission['meter_reading'] < 0), 0, submission['meter_reading'])
submission.to_csv('solution.csv.gz',index=False,compression='gzip')
submission.head()

,row_id,meter_reading
0,0,33.650735
1,1,20.133963
2,2,5.319478
3,3,17.110090
4,4,61.721774


Atualmente esta solução tem uma RMSLE de 1.14 no leaderboard do Kaggle e está entre as melhores 29%. A melhor solução publicada até o momento tem um RMSLE de 1.05.